In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import multilabel_confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError
from sklearn.metrics import mean_squared_error, r2_score
import pickle

2024-04-03 16:52:56.234661: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 16:52:56.522504: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 16:52:56.522559: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 16:52:56.523654: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 16:52:56.641805: I tensorflow/core/platform/cpu_feature_g

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [2]:
path = '../'
path_BD = 'BD_1/'
path_models = 'models/'

In [3]:
import time
timer = time.time()

In [ ]:
def define_model():
  # Defining a global average pooling layer and a classfication layer
  # global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  avg_layer = Lambda(lambda x: tf.reduce_mean(x, axis=-1))

  # use the preprocessing method included with the model.
  preprocess_input = tf.keras.applications.vgg16.preprocess_input

  # Build a model by chaining together the data augmentation, rescaling,
  # base_model and feature extractor layers using the Keras Functional API.
  inputs = tf.keras.Input(shape=(334, 499, 3))

  x = preprocess_input(inputs)
  base_model = tf.keras.applications.EfficientNetB7(
                                            include_top=False,
                                            weights='imagenet',
                                            input_tensor=x,
                                            input_shape=None,
                                            pooling=False)

  x = base_model(x)

  outputs = x

  model = tf.keras.Model(inputs, outputs)
  model.summary()

  # Compile the model
  base_learning_rate = 0.0001
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
                loss='mse',
                metrics=['mse'])

  return model

In [ ]:
def load_database():
  file = path + path_BD + 'X_pack_3apr.pkl'
  with open(file, 'rb') as filepath:
      arquivos = pickle.load(filepath)

  X = arquivos["X"]

  return X

In [ ]:
X = load_database()
X = np.concatenate((X[:,:,:,np.newaxis],X[:,:,:,np.newaxis],X[:,:,:,np.newaxis]), axis = 3)
print(X.shape)

In [7]:
model = define_model()

2024-04-03 16:53:40.868319: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 16:53:40.943893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 16:53:40.944088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 334, 499, 3)]     0         
                                                                 
 tf.__operators__.getitem (  (None, 334, 499, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 334, 499, 3)       0         
 )                                                               
                                                                 
 efficientnetb7 (Functional  (None, 11, 16, 2560)      64097687  
 )                                                               
                                                                 
Total params: 64097687 (244.51 MB)
Trainable params: 63786960 (243.33 MB)
Non-trainable params: 310727 (1.19 MB)
______________

In [8]:
batch_size = 32
n_runs = int(X.shape[0]/batch_size)
sobra = X.shape[0]%batch_size
if (sobra != 0): n_runs += 1

In [10]:
X_transformed = []  # Use a list instead of a tensor

for i in range(n_runs):
    X_batch_transformed = model(X[i*batch_size:(i+1)*batch_size,:,:,:])
    X_transformed.append(X_batch_transformed)
    print(i)

# Convert the list to a NumPy array
X_transformed = np.concatenate(X_transformed, axis=0)

2024-04-03 16:53:51.016647: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-03 16:53:51.286186: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-04-03 16:53:51.438285: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [12]:
filepath = path + path_BD + 'X_EfficientNetB7.pkl'
import pickle
with open(filepath,"wb") as filepath:
    pickle.dump({"X_EfficientNetB7_encoded": X_transformed}, filepath)

In [13]:
import gc 
gc.collect()

0